## Import Necessary Libraries

In [121]:
import pandas as pd
import tensorflow as tf
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Convolution2D, MaxPooling2D, Flatten, GlobalAveragePooling2D
from keras import models, layers
from keras_preprocessing.image import load_img, array_to_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import  Adam
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [122]:
datagen = ImageDataGenerator()

In [123]:
train_path =  './data/fruits-360_dataset/fruits-360/Training'
test_path = './data/fruits-360_dataset/fruits-360/Test'
val_path = './data/fruits-360_dataset/fruits-360/Validation'

In [124]:
golden_1_train_path = "./data/fruits-360_dataset/fruits-360/Training/Apple Golden 1"

In [125]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2, 
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [126]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (200, 200),
    batch_size = 32,
    color_mode = 'rgb',
    class_mode = 'categorical')

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size = (200, 200),
    batch_size = 32,
    color_mode = 'rgb',
    class_mode = 'categorical')

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size = (200, 200),
    batch_size = 32,
    color_mode = 'rgb',
    class_mode = 'categorical')

Found 4711 images belonging to 6 classes.
Found 1751 images belonging to 6 classes.
Found 523 images belonging to 6 classes.


In [127]:
train_class_names = list(train_generator.class_indices.keys())
train_class_names

['Apple Braeburn',
 'Apple Crimson Snow',
 'Apple Golden',
 'Apple Granny Smith',
 'Apple Pink Lady',
 'Apple Red Delicious']

In [128]:
test_class_names = list(test_generator.class_indices.keys())
test_class_names

['Apple Braeburn',
 'Apple Crimson Snow',
 'Apple Golden',
 'Apple Granny Smith',
 'Apple Pink Lady',
 'Apple Red Delicious']

In [129]:
val_class_names = list(val_generator.class_indices.keys())
val_class_names

['Apple Braeburn',
 'Apple Crimson Snow',
 'Apple Golden',
 'Apple Granny Smith',
 'Apple Pink Lady',
 'Apple Red Delicious']

## Begin Modeling

In [130]:
# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [131]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

In [132]:
print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(32, 120000)
(32, 120000)
(32, 120000)


In [133]:
train_y = np.reshape(train_labels[:,0], (32,1))
test_y = np.reshape(test_labels[:,0], (32,1))
val_y = np.reshape(val_labels[:,0], (32,1))

In [134]:
np.random.seed(3)
baseline_model = models.Sequential()
baseline_model.add(layers.Dense(20, activation='relu', input_shape=(120000,))) # 2 hidden layers
baseline_model.add(layers.Dense(7, activation='relu'))
baseline_model.add(layers.Dense(5, activation='relu'))
baseline_model.add(layers.Dense(1, activation='sigmoid'))

In [135]:
baseline_model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

histoire = baseline_model.fit(train_img,
                    train_y,
                    epochs = 30,
                    batch_size = 32,
                    validation_data = (val_img, val_y))

Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 0.5605 - accuracy: 0.7500

2022-12-05 17:58:02.092902: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-12-05 17:58:02.262618: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 367ms/step - loss: 0.5605 - accuracy: 0.7500 - val_loss: 6.5075 - val_accuracy: 0.0312
Epoch 2/30
1/1 [==============================] - 0s 116ms/step - loss: 5.2685 - accuracy: 0.2500 - val_loss: 0.5758 - val_accuracy: 0.9688
Epoch 3/30
1/1 [==============================] - 0s 99ms/step - loss: 0.6219 - accuracy: 0.7500 - val_loss: 0.3995 - val_accuracy: 0.9688
Epoch 4/30
1/1 [==============================] - 0s 105ms/step - loss: 0.5193 - accuracy: 0.8125 - val_loss: 0.2244 - val_accuracy: 0.9688
Epoch 5/30
1/1 [==============================] - 0s 99ms/step - loss: 0.5102 - accuracy: 0.7500 - val_loss: 0.2779 - val_accuracy: 0.9688
Epoch 6/30
1/1 [==============================] - 0s 98ms/step - loss: 0.4978 - accuracy: 0.7500 - val_loss: 0.2596 - val_accuracy: 0.9688
Epoch 7/30
1/1 [==============================] - 0s 99ms/step - loss: 0.4905 - accuracy: 0.7500 - val_loss: 0.2592 - val_accuracy: 0.9688
Epoch 8/30
1/1 [===================

In [136]:
# sz = 200

# # Initializing the CNN
# model1 = Sequential()

# # First convolution layer and pooling
# model1.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 3), activation='relu'))
# model1.add(MaxPooling2D(pool_size = (2, 2)))

# # Second convolution layer and pooling
# model1.add(Convolution2D(32, (3, 3), activation='relu'))
# # input_shape is going to be the pooled feature maps from the previous convolution layer
# model1.add(MaxPooling2D(pool_size=(2, 2)))

# # Flattening the layers
# model1.add(Flatten(input_shape = (28,28)))

# # Adding a fully connected layer
# model1.add(Dense(units=96, activation='relu'))
# model1.add(Dropout(0.40))
# model1.add(Dense(units=32, activation='relu'))
# model1.add(Dense(units=24, activation='softmax')) # softmax for more than 2

In [137]:
# model1.summary()

In [138]:
# model1.compile(optimizer='adam',
#                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#                metrics=['accuracy'])

In [139]:
# history = model1.fit(
#     train_generator,
#     validation_data = test_generator,
#     epochs = 5
# )